In [8]:
import os
import sys
import time
import openai
import jsonlines

### keys have to be in the environment
openai.api_type = "azure"
openai.api_key = "7d86bd0519b4468f8a1bf711708f8f11"
openai.api_version = "2023-03-15-preview"
openai.api_base = "https://girisimgpt.openai.azure.com/"

In [10]:
# You are an assistant designed to work with data in a pandas DataFrame.
# The DataFrame contains 1 million rows of data, and you need to create a df.loc filter based on user's queries.
# The 'IslemTarih' column ranges from 2014 to 2023, and the 'Marka' column represents company names.
# You have been given a small portion of this DataFrame as a sample.
# You should provide an initial response to the user's query without revealing the code steps.
# You should create a filter, but only provide the code without any explanation in the "FILTRE" section.

systemPrompt = {"role":"system","content":"""Sen bir ödeal yardımcı botusun. 1 milyon datalık bir pandas dataframe'i var ve kullanıcın sorduğu soruya göre df.loc filtresi oluştur. Oluşturduğun filtreyi sonda print ile çalıştır
IslemTarih sütunu 2014'ten 2023'e kadar uzanmaktadır. Marka sütunu ise şirket adlarını belirtmektedir. Sana bu dataframe'in küçük bir parçasını vereceğim:
                	id	Marka	IsyeriTipi	UyeDurum	UyeAdres	Sehir	UyeAktivasyonTarih	IslemID	IslemTarih	IslemTutar	Tercih	Versiyon	SurumTarih
758694	301029860	Oden o- maden	Tüzel Şirket	Pasif	ESENTEPE MAH. KORE ŞEHİTLERİ CAD. 4 -6 20 ŞİŞL...	İstanbul	2018-09-14 10:19:24	403271774	2019-02-07 13:54:40	20.0	ÖKC	NaN	NaN
856781	301031272	Blas burillas	Şahıs Şirketi	Aktif	CEVİZLİ MAH. MUSTAFA KEMAL CAD. HUKUKÇULAR TOW...	İstanbul	2018-11-21 13:29:14	493902770	2022-12-30 20:30:00	215.0	ÖKC	NaN	NaN
331533	301024729	Aüfe ass büfe	Tüzel Şirket	Pasif	MEHMET NESİH ÖZMEN MAH. KESTANE SK. AS CAFE NO...	İstanbul	2022-12-10 15:38:46	553180719	2023-06-22 15:14:00	41.5	ÖKC	NaN	NaN
165071	301012873	Eşti ekd. şti	Tüzel Şirket	Aktif	ERZENE MAH. KAZIM KARABEKİR CAD. NO: 4 G BORNO...	İzmir	2022-08-16 08:48:29	479806278	2022-10-19 17:08:00	195.0	ÖKC	NaN	NaN
29046	301003354	Aida aka gida	Tüzel Şirket	Aktif	KOCATEPE MAH. ŞEHİR PARKI CAD NO: 105 İÇ KAPI ...	İstanbul	2014-09-10 12:54:01	496940306	2023-01-10 08:50:00	7.5	ÖKC	NaN	NaN

Bugünün tarihi 7 Kasım 2023.
Kullanıcıya bir filtre oluşturmadan ilk önce sorduğu soruya aşağıdaki sırada cevap ver:

CEVAP: [kullanıcıya verilecek cevap] (burada kodun aşamalarını söyleme)

FILTRE:
[pandas kodu] (burada sadece kod olsun asla ama asla cevap veya yönerge olmasın!!! oluşturduğun filtreyi sonda print ile çalıştır)"""}

In [11]:
history = []
# Append the system's initial prompt to the conversation history.
history.append(systemPrompt)
# Append the assistant's greeting message to the conversation history.
history.append({"role": "assistant", "content":"Merhaba ben ÖDEAL asistan botuyum size nasıl yardımcı olabilirim"})
# User's query about total expenses between specific dates.
history.append({"role": "user", "content": "26 ekim 2021 ile 31 ekim 2021 arasında ne kadar para harcandı"})
# Assistant's response with the answer and code to calculate total expenses.
history.append({"role": "assistant", "content":"""Merhaba size şimdi yardımcı olacağım. 26 ekim 2021 ve 31 ekim 2021 tarihleri arasında harcanan miktar aşağıda verilmiştir.

FILTRE:
print(df.loc[(df['IslemTarih'] >= '2021-10-26') & (df['IslemTarih'] <= '2021-10-31'), 'IslemTutar'].sum())"""})
history.append({"role": "user", "content": "Fenersaray şirketi olarak bana bir kampanya oluştur."})
history.append({"role": "assistant", "content": """Tabii ki! Müşterilerinize bağlılığını artırmak için aşağıda bir kampanya önerisi sunabilirim:

Kampanya Önerisi: "Sadık Müşteri İndirimi"
- Müşterilerinize özel bir indirim kampanyası başlatın.
- Belirli bir süre boyunca, müşterilerinizin belirli bir tutarın üzerindeki alışverişlerinde indirim uygulayın.
- Örneğin, 100 TL ve üzeri alışverişlerde %10 indirim uygulayabilirsiniz.
- Kampanya süresi boyunca müşterilerinize bu indirimden faydalanabileceklerini bildirin.

Bu kampanya müşterilerinizin sizinle daha sık alışveriş yapmasını teşvik edebilir ve bağlılıklarını artırabilir. Ayrıca, müşterilerinize özel indirimler sunarak onları değerli hissettirebilirsiniz.

Unutmayın, kampanya detaylarını müşteri tabanınıza ve iş stratejinize göre özelleştirebilirsiniz.

FILTRE:
print(df.loc[df['Marka']=="Fenersaray"].IslemTutar.sum())
"""})



history.append({"role": "user", "content": "Kaç tane pasif şirket var?"})
history.append({"role": "assistant", "content": """Mevcut veri setinde kaç tane pasif şirket olduğunu bulmak için aşağıdaki kodu kullanabilirsiniz. Bu kod, 'UyeDurum' sütununda "Pasif" olan şirketleri filtreler ve 'Marka' sütununda benzersiz değerleri sayar. Bu şekilde, pasif şirketlerin sayısını elde edebilirsiniz.
FILTRE:
print(df.loc[df['UyeDurum'] == 'Pasif', 'Marka'].nunique())
"""})

# Testing model for some sample questions

#history.append({"role": "user", "content": "22 ocak 2019 ile 6 şubat 2020 arasında ne kadar para harcandı"})
#history.append({"role": "user", "content": "Kaç tane aktif şirket var?"})
#history.append({"role": "user", "content": "Neden mobil POS cihazı kullanmalıyım"})
#history.append({"role": "user", "content": "Ben unlu mamuller alanında çalışan bir şirketim. Sabah saatlerinde satılmayan börekler bayatlıyor. Nasıl bu satışı arttırırım?"})
#history.append({"role": "user", "content": "Yarınki tahmini satış miktarım son 24 saatteki satışa göre tahmini ne kadar olur?"})
#history.append({"role": "user", "content": "Bugünün tarihinde 30 gün öncesine kadar toplam kaç işlem yapıldı"})
#history.append({"role": "user", "content": "Kaç tane şehir var?"})
#history.append({"role": "user", "content": "Toplamda en çok para kazanmış marka hangisi"})
history.append({"role": "user", "content": "Son bir ayda yapılan işlem miktarına göre sonraki ay ne kadar işlem olabileceğini tahmin et"})

In [12]:
# Use the OpenAI API to generate a response based on the conversation history.
response = openai.ChatCompletion.create(engine="girisimgpt", messages=history, temperature=0)

In [16]:
# Print the content of the response generated by the OpenAI API.
print(response.choices[0].message.content)

Son bir ayda yapılan işlem sayısına göre sonraki ay ne kadar işlem olabileceğini tahmin etmek için aşağıdaki adımları takip edebilirsiniz:

1. Son bir ayın başlangıç ve bitiş tarihlerini belirleyin. Örneğin, 1 Ekim 2023 - 31 Ekim 2023.
2. Bu tarih aralığına göre veri setini filtreleyin ve sadece bu tarihler arasındaki işlemleri içeren bir alt veri seti oluşturun.
3. Oluşturduğunuz alt veri setindeki işlem sayısını hesaplayın.
4. Sonraki ayın başlangıç ve bitiş tarihlerini belirleyin. Örneğin, 1 Kasım 2023 - 30 Kasım 2023.
5. Sonraki ayın işlem sayısını tahmin etmek için, son bir aydaki işlem sayısını sonraki ayın gün sayısına oranlayın ve bu oranı sonraki ayın işlem sayısı tahminine uygulayın.

Örneğin, son bir ayda 1000 işlem yapıldı ve sonraki ay 30 gün olduğunu varsayarsak, tahmini işlem sayısı:

FILTRE:
last_month_transactions = df.loc[(df['IslemTarih'] >= '2023-10-01') & (df['IslemTarih'] <= '2023-10-31')]
last_month_transaction_count = len(last_month_transactions)

next_month_day

In [17]:
# Assign the content of the response to the variable 'x'.
x = response.choices[0].message.content

In [19]:
# Try to extract code from 'x':
# 1. First, attempt to split the content based on "FILTRE:" and select the part after that substring.
# 2. If the first attempt fails, check if the content contains "import" and extract the code from there.
# 3. If both attempts fail, print "kod yok" (no code).

try:
    xlist = x.split("FILTRE:")[1]
except:
    try:
        xind = x.index("import")
        xlist = x[xind:]
    except:
        print("kod yok")

In [29]:
# Read data from a CSV file named "hackathon_data.csv" while ignoring low memory warnings.
# Drop specific columns by their index: the first, 11th, and 12th columns.
# Convert the "IslemTarih" column to datetime format.

import pandas as pd
data = pd.read_csv("hackathon_data.csv", low_memory=False)
data.drop(data.columns[[0, 11, 12]], axis=1, inplace=True)
df = data
df["IslemTarih"] = pd.to_datetime(df["IslemTarih"])
data.head()

,Marka,IsyeriTipi,UyeDurum,UyeAdres,Sehir,UyeAktivasyonTarih,IslemID,IslemTarih,IslemTutar,Tercih
0,Kzar kü pazar,Tüzel Şirket,Aktif,KALE MAH. GAZİ CAD. 82 C İLKADIM/ SAMSUN,Samsun,2015-09-08 15:08:23,401003136,2015-09-18 14:35:04,5.00,CeptePos
1,Kzar kü pazar,Tüzel Şirket,Aktif,KALE MAH. GAZİ CAD. 82 C İLKADIM/ SAMSUN,Samsun,2015-09-08 15:08:23,403526702,2019-04-13 13:31:00,32.70,ÖKC
2,Kzar kü pazar,Tüzel Şirket,Aktif,KALE MAH. GAZİ CAD. 82 C İLKADIM/ SAMSUN,Samsun,2015-09-08 15:08:23,403529006,2019-04-13 19:30:00,998.05,ÖKC
3,Kzar kü pazar,Tüzel Şirket,Aktif,KALE MAH. GAZİ CAD. 82 C İLKADIM/ SAMSUN,Samsun,2015-09-08 15:08:23,403530803,2019-04-14 13:52:00,105.75,ÖKC
4,Kzar kü pazar,Tüzel Şirket,Aktif,KALE MAH. GAZİ CAD. 82 C İLKADIM/ SAMSUN,Samsun,2015-09-08 15:08:23,403531107,2019-04-14 15:09:00,163.00,ÖKC


In [33]:
# Execute each line from 'xlist' within a loop.

for i in xlist.split("\n"):
    try:
        exec(i)
    except:
        # Attempt to modify the "IslemTarih" column and re-execute the lines from 'xlist.'
        try:
            df["IslemTarih1"] = df["IslemTarih"].copy()
            df["IslemTarih"] = df["IslemTarih"].dt.date
            for j in xlist.split("\n"):
                exec(j)
            df["IslemTarih"] = df["IslemTarih1"].copy()        
        except:
            # Print an error message if both attempts fail.

            print("timeerrortimeerror")
        break
        

14797
